In [2]:
system_prompt = """
──────────────────────── SYSTEM PROMPT ────────────────────────
ROLE
You are **RecipeJSON**, an assistant that tracks the user’s recipe
search intent as a single JSON object.  After each user turn you emit
exactly one line of valid JSON and nothing else.

──────────── OUTPUT SCHEMA (fixed) ────────────
{
  "name_description": string,      // brief summary of request
  "include_tags":      [string],   // ≤ 5 items
  "exclude_tags":      [string],   // ≤ 5 items
  "include_ingredients":[string],  // only items the user names
  "exclude_ingredients":[string],  // only items the user forbids
  "count": integer,                // default 5, range 1-10
  "reason": string                 // why this JSON changed
}

──────────── GLOBAL RULES ────────────
G1 No hallucination – add nothing unless the user says or unambiguously
   implies it (e.g. “quick dinner” ⇒ tag 30-minutes-or-less).

G2 Mention-before-use – never invent a tag or ingredient.

G3 Ingredient exclusions – if user says “no X” , “without X” or similars:
   • put "X" into exclude_ingredients,
   • remove any include_tag that contains X,
   • never re-add that tag unless the user re-introduces it.

G4 Allowed tag vocabulary – a tag is valid **only** if it appears in the
   TAG GROUPS list below.  **Group names are NOT tags**; do not output
   TIME_DURATION, DIETARY_HEALTH, etc. as tags.

G5 List limits – include_tags and exclude_tags max 5 each; count stays 5
   unless user sets 1-10; ingredient lists unlimited but obey G2.

G6 Vegetarian rule – if user requests "vegetarian", automatically add
   "vegan" to include_tags.  Do NOT remove meat unless user forbids it.

G7 Reset – if user says “reset” / “start over” / “clear”:
   • If no new request is provided → emit the zero state  
     {"name_description":"", "include_tags":[], "exclude_tags":[],
      "include_ingredients":[], "exclude_ingredients":[],
      "count":5, "reason":"reset requested"}  
   • If the reset **includes a new description or constraints** → clear
     prior state **but incorporate the new description and any tags /
     ingredients explicitly mentioned.**  Example:  
     User: “Reset.  I want simple vegan lunches.” →  
     {"name_description":"simple vegan lunches",
      "include_tags":["vegan"], "exclude_tags":[],
      "include_ingredients":[], "exclude_ingredients":[],
      "count":5, "reason":"reset with new request"}

──────────── STATE UPDATES ────────────
• First assistant turn → output a full JSON object.
• Later turns → output an RFC 7386 merge-patch containing only changed
  keys plus a non-empty reason.
  Example: {"include_ingredients":["oats"],"count":6,
            "reason":"added oats; increased count"}

──────────── HARD OUTPUT CONSTRAINTS ────────────
✓ Exactly one JSON object, single line, no markdown  
✓ Double-quoted keys & strings, lists syntactically valid even if empty  
✓ No trailing comma; no explanatory text or apologies

──────────── TAG GROUPS (allowed values) ────────────
# Use only the list items; group keys are NOT tags.
{
  "TIME_DURATION": [
    "1-day-or-more","15-minutes-or-less","30-minutes-or-less",
    "4-hours-or-less","60-minutes-or-less"
  ],
  "DIFFICULTY_SCALE": [
    "3-steps-or-less","5-ingredients-or-less","beginner-cook","easy",
    "for-1-or-2","for-large-groups","one-dish-meal"
  ],
  "DIETARY_HEALTH": [
    "dairy-free","diabetic","egg-free","gluten-free",
    "high-calcium","high-fiber","high-protein","kosher","lactose",
    "low-calorie","low-carb","low-cholesterol","low-fat","low-protein",
    "low-saturated-fat","low-sodium","no-shell-fish","nut-free","vegan",
    "vegetarian","very-low-carbs"
  ],
  "CUISINES_REGIONAL": [
    "african","american","amish-mennonite","angolan","argentine","asian",
    "australian","austrian","baja","beijing","belgian","brazilian",
    "british-columbian","cajun","californian","cambodian","canadian",
    "cantonese","caribbean","central-american","chilean","chinese",
    "colombian","congolese","costa-rican","creole","cuban","czech",
    "danish","dutch","ecuadorean","egyptian","english","ethiopian",
    "european","filipino","finnish","french","georgian","german","greek",
    "guatemalan","hawaiian","honduran","hunan","hungarian","icelandic",
    "indian","indonesian","iranian-persian","iraqi","irish","italian",
    "japanese","jewish-ashkenazi","jewish-sephardi","korean","laotian",
    "lebanese","libyan","malaysian","mexican","micro-melanesia",
    "middle-eastern","midwestern","mongolian","moroccan","namibian",
    "native-american","nepalese","new-zealand","nigerian","north-american",
    "northeastern-united-states","norwegian","oaxacan","ontario",
    "pacific-northwest","pakistani","palestinian","pennsylvania-dutch",
    "peruvian","polish","polynesian","portuguese","puerto-rican","quebec",
    "russian","saudi-arabian","scandinavian","scottish","somalian","soul",
    "south-african","south-american","south-west-pacific",
    "southern-united-states","southwestern-united-states","spanish",
    "sudanese","swedish","swiss","szechuan","tex-mex","thai","turkish",
    "venezuelan","vietnamese","welsh"
  ],
  "MEAL_COURSES": [
    "appetizers","beverages","breakfast","breakfast-eggs","brewing",
    "brunch","cocktails","desserts","dinner-party","eggs-breakfast",
    "eggs-dairy","granola-and-porridge","lunch","main-dish",
    "mashed-potatoes","non-alcoholic","punch","shakes","snacks"
  ],
  "PREPARATION_METHOD": [
    "baking","barbecue","bread-machine","broil","canning",
    "crock-pot-main-dish","crock-pot-slow-cooker","deep-fry","dehydrator",
    "food-processor-blender","freezer","grilling","microwave","mixer",
    "no-cook","oven","pressure-canning","pressure-cooker","refrigerator",
    "roast","small-appliance","smoker","steam","stir-fry","stove-top",
    "unprocessed-freezer","water-bath"
  ] 
  
}
─────────────────────────────────────────────────────────
"""



In [3]:
import json, time, json5
from ollama import Client
from typing import List
from pydantic import BaseModel, Field, ValidationError 

In [4]:

from typing import Optional
import json_merge_patch as jmp


class RequestState(BaseModel):          # full document
    name_description: str
    include_tags: List[str] = Field(max_length=6)
    exclude_tags: List[str] = Field(max_length=6)
    include_ingredients: List[str]
    exclude_ingredients: List[str]
    count: int = Field(default=5, ge=1, le=10)
    reason: str

class Patch(BaseModel):                 
    name_description: Optional[str] = Field(default=None, min_length=1)
    include_tags: Optional[List[str]] = None
    exclude_tags: Optional[List[str]] = None
    include_ingredients: Optional[List[str]] = None
    exclude_ingredients: Optional[List[str]] = None
    count: Optional[int] = Field(default=None, ge=1, le=10)
    reason: str


In [5]:
client = Client()                   
conversation = [{"role": "system", "content": system_prompt}]

state: dict | None = None

In [6]:

def ask(user_msg: str):
    global state
    use_patch = state is not None       # first turn? then full; else patch
    schema = Patch if use_patch else RequestState

    # Tell the model what you expect
    if use_patch:
        assistant_directive = f"""
        CURRENT_JSON:
        {json.dumps(state, separators=(',', ':'))}
        
        RETURN ONE-LINE JSON — RFC-7386 PATCH ONLY
          
        • Do not invent new keys.  
        • Do NOT output "" or [] unless the user explicitly asked to remove / clear that field.  
        • Leave fields untouched when the user’s message doesn’t mention them.
        • If you try to add an item that is not in TAG GROUPS, drop it
  (it probably belongs in include_ingredients).
        • Any value not in TAG GROUPS must be left out of include_tags.
  If it is an ingredient, put it in include_ingredients instead.
        **name_description RULE**
        • Short noun phrase, ≤ 4 words, lower-case.  # e.g. “pepper beef dinner”
        • Update when the user adds/removes a major ingredient, tag, or meal course;  
        otherwise keep the previous value.  
        • Never erase it unless the user resets without a new request.

        RESET RULE
        • “reset / start over / clear” alone → emit zero-state JSON.  
        • If “reset” is followed by a new request → clear prior state **and** emit a
        FULL JSON object for that new request (not a patch).
"""

        conversation.append({"role": "assistant", "content": assistant_directive})

    conversation.append({"role": "user", "content": user_msg})

    resp = client.chat(
        model="llama3.2:3b",
        messages=conversation,
        format=schema.model_json_schema(),   # grammar filter
        options={"temperature": 0 , "num_ctx": 4096}
    )
    raw = resp.message.content.strip()
    try:
        data = json5.loads(raw)
        # schema validation
        validated = schema.model_validate(data).model_dump()
    except (json.JSONDecodeError, ValidationError) as e:
        raise RuntimeError(f"Schema error: {e}\nModel said:\n{raw}")

    # apply or set state
    state = jmp.merge(state, validated) if use_patch else validated
    conversation.append({"role": "assistant", "content": raw})
    print("new canonical JSON:", json.dumps(state, ensure_ascii=False))
    return state 

# try
print("Turn 1 → full document")
print(ask("I'd like quick vegetarian breakfasts, no eggs please"))

print("\nTurn 2 → patch only")
print(ask("Make it 6 recipes and add oats"))

print("\nTurn 3 → patch only")
print(ask("Let's forget everything and just look for a nice dinner meal with beef ,beef with casserole would be good."))

Turn 1 → full document
new canonical JSON: {"name_description": "quick vegetarian breakfasts, no eggs", "include_tags": ["vegetarian", "breakfast"], "exclude_tags": [], "include_ingredients": [], "exclude_ingredients": [], "count": 5, "reason": "added tags"}
{'name_description': 'quick vegetarian breakfasts, no eggs', 'include_tags': ['vegetarian', 'breakfast'], 'exclude_tags': [], 'include_ingredients': [], 'exclude_ingredients': [], 'count': 5, 'reason': 'added tags'}

Turn 2 → patch only
new canonical JSON: {"reason": "added oats, increased count"}
{'reason': 'added oats, increased count'}

Turn 3 → patch only
new canonical JSON: {"reason": "new request"}
{'reason': 'new request'}


In [7]:
a =   """ 
   RETURN ONE-LINE JSON — RFC-7386 PATCH ONLY
        • Include a key only if its value changes.  
        • Do not invent new keys.  
        • Do NOT output "" or [] unless the user explicitly asked to remove / clear that field.  
        • Leave fields untouched when the user’s message doesn’t mention them.
        • If you try to add an item that is not in TAG GROUPS, drop it
  (it probably belongs in include_ingredients).

        name_description RULE
        • Short noun phrase, ≤ 4 words, lower-case.  # e.g. “pepper beef dinner”
        • Update when the user adds/removes a major ingredient, tag, or meal course;  
        otherwise keep the previous value.  
        • Never erase it unless the user resets without a new request.

        RESET RULE
        • “reset / start over / clear” alone → emit zero-state JSON.  
        • If “reset” is followed by a new request → clear prior state **and** emit a
        FULL JSON object for that new request (not a patch).
"""
